In [ ]:
def dist_vec(vec_a, vec_b):
    return np.sqrt(np.sum(np.power((vec_a - vec_b),2)))

def k_means_clustering_test(data, k):
    m, n = data.shape
    centroids = data.take(np.random.choice(m,k,replace=False),axis=0).reset_index(drop=True)
    # print(centroids)

    distance = []
    for i in range(k):
        distance.append(1)

    groups = []

    while sum(distance) != 0.0 :

        groups = []
        for i in range(k):
            groups.append(pd.DataFrame(columns=list(data.columns)))

        for i, r in data.iterrows():
            p = np.matrix(r)
            distances = [dist_vec(p, np.matrix(centroids.iloc[c,:])) for c in range(centroids.shape[0])]
            min_dis = np.argmin(distances)
            groups[min_dis].loc[len(groups[min_dis])] = list(p.flat)

        new_centroids = pd.DataFrame(columns=list(data.columns))
        for i, g in enumerate(groups):
            mid = np.matrix(np.mean(g, axis=0))
            dis = dist_vec(mid, np.matrix(centroids.iloc[i,:]))
            if  dis < 1e-2:
                dis = 0.0

            distance[i] = dis
            new_centroids.loc[len(new_centroids)] = list(mid.flat)
            # print("length: ", len(g))
            # print(dis)
        centroids = new_centroids

    # print(centroids)
    return centroids, groups

k_mean_result = k_means_clustering_test(x, 10)

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans

def k_means(data):
    sse = []
    for kk in range(1,15):
        estimator = KMeans(n_clusters=kk)
        estimator.fit(data)
        sse.append(estimator.inertia_)
    X = range(1,15)
    plt.xlabel('k')
    plt.ylabel('SSE')
    plt.plot(X,sse,'o-')
    plt.show()

k_means(x)

In [ ]:
def best_k(dataset, size):
    # Split data to train and valid
    train, valid = dataset.iloc[:size, :], np.array(dataset.iloc[size:, :])
    k=1

    # Calculate centroids and cluster
    result_centroids, result_cluster = k_means_clustering(train, k)
    # Calculate loss function
    distance_cur = loss_dis(valid, result_centroids, k)

    dist_diff = [] # Distance between previous loss and current loss
    for kk in range(2,15):

        distance_bef = distance_cur
        result_centroids, result_cluster = k_means_clustering(train, kk)
        distance_cur = loss_dis(valid, result_centroids, kk)
        dis = distance_bef - distance_cur
        dist_diff.append(dis)
        # When current loss difference is much smaller than previous, pup out previous k
        # According to Elbow method, when the optimization effect becomes insignificant, the optimal solution appears
        if kk > 3:
            if 0.9 < dist_diff[-1]/dist_diff[-2] < 1:
                k=kk-1
                break
            if dist_diff[-2]/dist_diff[-3] > 1:
                if dist_diff[-1]/dist_diff[-3] > 0.9:
                    k=kk-2
                    print(k)
                    break

    X = range(2,kk+1)
    plt.xlabel('k')
    plt.ylabel('dist_diff')
    plt.plot(X,dist_diff,'o-')
    plt.show()
    return k

best_k(x, 4000)